   * https://debezium.io/documentation/reference/stable/connectors/sqlserver.html: Debezium connector for SQL Server
   * https://github.com/debezium/debezium-examples/tree/main/tutorial#using-sql-server
      * https://github.com/debezium/debezium-examples/blob/main/tutorial/debezium-sqlserver-init/inventory.sql

In [2]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-sqlserver -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"debezium-connector-sqlserver","namespace":"kafka-main-cluster"},"spec":{"class":"io.debezium.connector.sqlserver.SqlServerConnector","config":{"converter.type":"avro","database.dbname":"testDB","database.encrypt":false,"database.history.kafka.bootstrap.servers":"main-kafka-bootstrap:9092","database.history.kafka.topic":"sqlserver-cdc-tables","database.hostname":"mssqlserver-2022.sqlserver.svc","database.names":"testDB","database.password":"jRaF7SUPhgJ6iDkQZCjn","database.port":1433,"database.server.name":"mssqlserver-2022.sqlserver.svc","database.user":"sa","decimal.handling.mode":"double","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://schema-re

# Get connector errors

In [48]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-sqlserver -o jsonpath='{.status.conditions}' | jq

[
  {
    "lastTransitionTime": "2024-01-30T11:34:35.548400835Z",
    "message": "PUT /connectors/debezium-connector-sqlserver/config returned 400 (Bad Request): Connector configuration is invalid and contains the following 1 error(s):\nUser sa does not have access to CDC schema in the following databases: test. This user can only be used in initial_only snapshot mode\nYou can also find the above list of errors at the endpoint `/connector-plugins/{connectorType}/config/validate`",
    "reason": "ConnectRestException",
    "status": "True",
    "type": "NotReady"
  }
]


   * https://debezium.io/documentation/reference/stable/connectors/sqlserver.html#setting-up-sqlserver
      * "Example: Enabling a SQL Server database for the CDC template"   
```sql
USE test

EXEC sys.sp_cdc_enable_db
EXEC sys.sp_cdc_help_change_data_capture
```


# Get all Kafka Connect connectors
   * https://docs.confluent.io/platform/current/connect/references/restapi.html

In [50]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/ | jq

[
  {
    "class": "io.debezium.connector.mongodb.MongoDbConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.mysql.MySqlConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.oracle.OracleConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.postgresql.PostgresConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.sqlserver.SqlServerConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorCheckpointConnector",
    "type": "source",
    "version": "3.6.1"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorHeartbeatConnector",
    "type": "source",
    "version": "3.6.1"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorSourceConnector",
    "type": "source",
    "version": "3.6.1"
  }
]


In [34]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connectors/?expand=status | jq

{
  "debezium-connector-postgres": {
    "status": {
      "name": "debezium-connector-postgres",
      "connector": {
        "state": "RUNNING",
        "worker_id": "my-connect-connect-0.my-connect-connect.kafka-main-cluster.svc:8083"
      },
      "tasks": [
        {
          "id": 0,
          "state": "RUNNING",
          "worker_id": "my-connect-connect-0.my-connect-connect.kafka-main-cluster.svc:8083"
        }
      ],
      "type": "source"
    }
  }
}


In [26]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/source/config/validate  | jq

{
  "error_code": 405,
  "message": "HTTP 405 Method Not Allowed"
}


In [35]:
!curl -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/source/config/validate  | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    58  100    58    0     0  23947      0 --:--:-- --:--:-- --:--:-- 29000
{
  "error_code": 405,
  "message": "HTTP 405 Method Not Allowed"
}
